In [1]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [6]:
from openai import OpenAI
api_key = "AP-KEY"

client = OpenAI(api_key=api_key)

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-3.5-turbo-1106",
)
show_json(assistant)

{'id': 'asst_LvWg4L8f1mbrtHiXba5ZkBtT',
 'created_at': 1700404112,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

In [7]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_gVUkxrojB1KX8f1VoGVxfEBg',
 'created_at': 1700404117,
 'metadata': {},
 'object': 'thread'}

In [8]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)
show_json(message)

{'id': 'msg_91xbmpkOyPpwEY0QHzl8h3gR',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1700404121,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_gVUkxrojB1KX8f1VoGVxfEBg'}

In [9]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_tLqs2wBa6UXQSJDjNrRq8EHV',
 'assistant_id': 'asst_LvWg4L8f1mbrtHiXba5ZkBtT',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1700404125,
 'expires_at': 1700404725,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_gVUkxrojB1KX8f1VoGVxfEBg',
 'tools': []}

In [10]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [11]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_tLqs2wBa6UXQSJDjNrRq8EHV',
 'assistant_id': 'asst_LvWg4L8f1mbrtHiXba5ZkBtT',
 'cancelled_at': None,
 'completed_at': 1700404126,
 'created_at': 1700404125,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1700404125,
 'status': 'completed',
 'thread_id': 'thread_gVUkxrojB1KX8f1VoGVxfEBg',
 'tools': []}

In [12]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_xkDhPvds2DMCagtVc0AX3vB1',
   'assistant_id': 'asst_LvWg4L8f1mbrtHiXba5ZkBtT',
   'content': [{'text': {'annotations': [],
      'value': 'Sure! The solution is x = 1.'},
     'type': 'text'}],
   'created_at': 1700404126,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_tLqs2wBa6UXQSJDjNrRq8EHV',
   'thread_id': 'thread_gVUkxrojB1KX8f1VoGVxfEBg'},
  {'id': 'msg_91xbmpkOyPpwEY0QHzl8h3gR',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
     'type': 'text'}],
   'created_at': 1700404121,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'thread_id': 'thread_gVUkxrojB1KX8f1VoGVxfEBg'}],
 'object': 'list',
 'first_id': 'msg_xkDhPvds2DMCagtVc0AX3vB1',
 'last_id': 'msg_91xbmpkOyPpwEY0QHzl8h3gR',
 'has_more': False}

In [13]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Could you explain this to me?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)

{'data': [{'id': 'msg_9t0dK4ZNnIc7YjyTSakTMte8',
   'assistant_id': 'asst_LvWg4L8f1mbrtHiXba5ZkBtT',
   'content': [{'text': {'annotations': [],
      'value': 'Sure! To solve for x, you need to isolate it on one side of the equation by performing the opposite operations. In this case, you subtract 11 from both sides to get 3x = 3, and then divide by 3 to find x = 1.'},
     'type': 'text'}],
   'created_at': 1700404385,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_e6LQjHzNJXfHirobPIAMe6fP',
   'thread_id': 'thread_gVUkxrojB1KX8f1VoGVxfEBg'}],
 'object': 'list',
 'first_id': 'msg_9t0dK4ZNnIc7YjyTSakTMte8',
 'last_id': 'msg_9t0dK4ZNnIc7YjyTSakTMte8',
 'has_more': False}

In [15]:
from openai import OpenAI

MATH_ASSISTANT_ID = assistant.id  # or a hard-coded ID like "asst-..."


def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

In [16]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    return thread, run


# Emulating concurrent user requests
thread1, run1 = create_thread_and_run(
    "I need to solve the equation `3x + 11 = 14`. Can you help me?"
)
thread2, run2 = create_thread_and_run("Could you explain linear algebra to me?")
thread3, run3 = create_thread_and_run("I don't like math. What can I do?")

# Now all Runs are executing...

In [17]:
import time

# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


# Wait for Run 1
run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

# Wait for Run 2
run2 = wait_on_run(run2, thread2)
pretty_print(get_response(thread2))

# Wait for Run 3
run3 = wait_on_run(run3, thread3)
pretty_print(get_response(thread3))

# Thank our assistant on Thread 3 :)
run4 = submit_message(MATH_ASSISTANT_ID, thread3, "Thank you!")
run4 = wait_on_run(run4, thread3)
pretty_print(get_response(thread3))

# Messages
user: I need to solve the equation `3x + 11 = 14`. Can you help me?
assistant: Sure! The solution is x = 1.

# Messages
user: Could you explain linear algebra to me?
assistant: Linear algebra is the branch of mathematics that deals with vector spaces and linear transformations, and it involves studying properties and operations with vectors and matrices.

# Messages
user: I don't like math. What can I do?
assistant: Try finding real-life applications of math that interest you, and keep practicing to strengthen your skills.

# Messages
user: I don't like math. What can I do?
assistant: Try finding real-life applications of math that interest you, and keep practicing to strengthen your skills.
user: Thank you!
assistant: You're welcome! If you have any more questions, feel free to ask.



In [20]:
# Upload the file
file = client.files.create(
    file=open(
        "C:/Users/Gianfranco Pizzuto/Downloads/data_GPA.txt",
        "rb",
    ),
    purpose="assistants",
)
# Update Assistant
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    file_ids=[file.id],
)
show_json(assistant)

{'id': 'asst_LvWg4L8f1mbrtHiXba5ZkBtT',
 'created_at': 1700404112,
 'description': None,
 'file_ids': ['file-4a1Enj8TeNXt5fA8BhpVw1pK'],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}, {'type': 'retrieval'}]}

In [21]:
thread, run = create_thread_and_run(
    "What is my dog's name?"
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: What is my dog's name?
assistant: Your dog's name is Sunny.

